In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
# llm.set_engine("davinci-codex")
output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-3.5-turbo')
print(output.content)

In [ ]:
help(ChatOpenAI)

In [ ]:
from langchain.schema import(
    SystemMessage,
    AIMessage,
    HumanMessage,
)

messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

## Caching LLM Responses
In-Memory Cache

In [ ]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI

llm = OpenAI(model_name='gpt-3.5-turbo-instruct')

In [ ]:
%%time
from langchain.cache import InMemoryCache
set_llm_cache(InMemoryCache())
prompt = 'Tell me a joke that a toddler can understand.'
llm.invoke(prompt)

In [ ]:
%%time
llm.invoke(prompt)

## SQLite Caching

In [ ]:
from langchain.cache import SQLiteCache
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

# Second request (cached, faster)
llm.invoke("Tell me a joke")

## LLM Streaming

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'
print(llm.invoke(prompt).content)

In [ ]:
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

## Prompt Templates

In [ ]:
from langchain.prompts import PromptTemplate
# from langchain_openai import ChatOpenAI
template = '''You are an experience virologist. Write a few sentences about the following virus "{virus}" in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

prompt = prompt_template.format(virus='hiv', language='german')
prompt

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)
output = llm.invoke(prompt)
print(output.content)

## ChatPromptTemplates

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage

chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

messages = chat_template.format_messages(n='10', area='Europe')
print(messages)

In [ ]:
# from langchain_openai import ChatOpenAI
output = llm.invoke(messages)
print(output.content)

## Simple Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI()
template = '''You are an experience virologist. Write a few sentences about the following virus {virus} in {language}.'''
prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})

print(output)

#### Another example with a single template value

In [ ]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'

prompt_template = PromptTemplate.from_template(template=template)

chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')
output = chain.invoke(country)
print(output['text'])

## Sequential Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain
# FIRST LLM
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)
prompt_template1 = PromptTemplate.from_template(
    template = 'You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)

chain1 = LLMChain(
    llm=llm1,
    prompt=prompt_template1
)

# SECOND LLM
llm2 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=1.2)
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
chain2 = LLMChain(
    llm=llm2,
    prompt=prompt_template2
)

overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

output = overall_chain.invoke('linear regression')

In [ ]:
print(output['output'])

## LangChain Agents in Action: Python REPL

In [ ]:
pip install -q langchain_experimental

## LangChain Tools: DuckDuckGp and Wikipedia

In [ ]:
# pip install -q duckduckgo-search

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()
output = search.invoke('Where was the Freddie Mercury born?')
print(output)

In [ ]:
search.name

In [ ]:
search.description

In [ ]:
from langchain.tools import DuckDuckGoSearchResults
search = DuckDuckGoSearchResults()
output = search.run('Freddie Mercury and Queen')
print(output)

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

wrapper = DuckDuckGoSearchAPIWrapper(region='de-de', max_results=3, safesearch='moderate')
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')
output = search.run('Berlin')

In [ ]:
print(output)

In [ ]:
import re
pattern = r'snippet: (.*?), title: (.*?), link: (.*?)\],'
matches = re.findall(pattern, output, re.DOTALL)

for snipper, title, link in matches:
    print(f'Snippet: {snipper}\nTitle: {title}\nLink: {link}\n')
    print('-' * 50)

In [ ]:
# pip install -q wikipedia

In [ ]:
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import WikipediaQueryRun

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=5000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

In [ ]:
wiki.invoke('Google Gemini')

## Creating a ReAct Agent

In [ ]:
# pip install langchainhub -q

In [ ]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import AgentExecutor, Tool, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

template = '''
Answer the following questions as best you can.
Questions: {q}
'''

prompt_template = PromptTemplate.from_template(template=template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you look up to a topic, country, or person on Wikipedia'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DucDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executer = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)


In [ ]:
question = 'Generate the first 20 numbers in the Fibonacci series.'
output = agent_executer.invoke({
    'input': prompt_template.format(q=question)
})

In [ ]:
print(output['input'])

In [ ]:
print(output['output'])

In [ ]:
question2 = 'Who is the current prime minister of the UK?'
output2 = agent_executer.invoke({
    'input': prompt_template.format(q=question2)
})

In [ ]:
question3 = 'Tell me about Napolean Bonaparte early life'
output3 = agent_executer.invoke({
    'input': prompt_template.format(q=question3)
})

In [ ]:
print(output3['input'])
print(output3['output'])